In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from connectivity import (plot_graph_circos, make_graph, 
                          graph_measures, z_scores,
                          similarity_between_subjects)
# NB: add more subjects in the data folder in neurolib package
from neurolib.utils.loadData import Dataset

In [ ]:
%%time
# NB: add more subjects in the data folder in neurolib package
ds = Dataset("gw")
columns = ['mean_degree', 'degree', 'closeness', 'betweenness',
           'mean_shortest_path', 'neighbor_degree', 'clustering_coefficient',
           'mean_clustering_coefficient', 'backbone', 'Cmat', 'Dmat']
df = pd.DataFrame(columns=columns)

for n, (cmat, dmat) in enumerate(zip([ds.Cmat] + ds.Cmats, [ds.Dmat] + ds.Dmats)):
    G = make_graph(cmat)
    G, gm = graph_measures(G) # , dmat
    gm['Cmat'] = cmat
    gm['Dmat'] = dmat
    df.loc[n] = gm
df.head()

In [ ]:
df.columns

# Correlations

In [ ]:
cmat_upper = np.array([df['Cmat'][i][np.triu_indices(80)].flatten().tolist() for i in range(1, 27)])
cmat_upper_corr = np.corrcoef(cmat_upper)
plt.figure(figsize = (7, 7))
_ = sns.heatmap(cmat_upper_corr, cmap='coolwarm', cbar=True, square=True)
cmat_upper_corr[np.triu_indices(26)].mean()

In [ ]:
dmat_upper = np.array([df['Dmat'][i][np.triu_indices(80)].flatten().tolist() for i in range(1, 27)])
dmat_upper_corr = np.corrcoef(dmat_upper)
plt.figure(figsize = (7, 7))
_ = sns.heatmap(dmat_upper_corr, cmap='coolwarm', cbar=True, square=True)
dmat_upper_corr[np.triu_indices(26)].mean()

# Inter-subject variability

In [ ]:
cmats = np.array([df.loc[i, 'Cmat'] for i in range(1, 27)])

In [ ]:
coefficient_of_variation = cmats.std(axis=0) / cmats.mean(axis=0)
np.fill_diagonal(coefficient_of_variation, 0)

In [ ]:
plt.figure(figsize = (18, 7))
heatmap = sns.heatmap(coefficient_of_variation, cmap='coolwarm', cbar=True, square=True, 
                      cbar_kws={"label": "Coefficient of Variation CV= mean/standard deviation"})
plt.xlabel('Nodes')
plt.ylabel('Nodes')
plt.show()

In [ ]:
G = make_graph(coefficient_of_variation/coefficient_of_variation.max())
plot_graph_circos(G, sc_threshold=0.7)

# Degree

In [ ]:
degree = np.array([df.loc[i, 'degree'].tolist() for i in range(27)])
plt.figure(figsize = (18, 7))
_ = sns.heatmap(degree, cmap='coolwarm', cbar=True, square=False, cbar_kws={"label": "Normalized node strength values 1/N-1"})
plt.xlabel('Nodes')
plt.ylabel('Subjects')
plt.show()

# Centrality

In [ ]:
degree = np.array([df.loc[i, 'betweenness'] for i in range(27)])
plt.figure(figsize = (18, 7))
heatmap = sns.heatmap(degree, cmap='coolwarm', cbar=True, square=False, 
                      cbar_kws={"label": "Normalized Betweenness Centrality"})
plt.xlabel('Nodes')
plt.ylabel('Subjects')
plt.show()

In [ ]:
degree = np.array([df.loc[i, 'closeness'] for i in range(27)])
plt.figure(figsize = (18, 7))
heatmap = sns.heatmap(degree, cmap='coolwarm', cbar=True, square=False, 
                      cbar_kws={"label": "Closeness Centrality"})
plt.xlabel('Nodes')
plt.ylabel('Subjects')
plt.show()

# Average shortest path length

In [ ]:
sns.violinplot(df['mean_shortest_path'].values)
plt.xlabel('Shortest path length')
plt.show()

# Average degree of the neighborhood

In [ ]:
degree = np.array([df.loc[i, 'neighbor_degree'] for i in range(27)])
plt.figure(figsize = (18, 7))
heatmap = sns.heatmap(degree, cmap='coolwarm', cbar=True, square=False, 
                      cbar_kws={"label": "Average strength of the neighborhood"}, vmin=75)
plt.xlabel('Nodes')
plt.ylabel('Subjects')
plt.show()

# Clustering Coefficient

In [ ]:
degree = np.array([df.loc[i, 'clustering_coefficient'] for i in range(27)])
plt.figure(figsize = (18, 7))
heatmap = sns.heatmap(degree, cmap='coolwarm', cbar=True, square=False, 
                      cbar_kws={"label": "Clustering Coefficient"})
plt.xlabel('Nodes')
plt.ylabel('Subjects')
plt.show()	

# Distribution of the graph measurements between participants

In [ ]:
df_z = z_scores(df)
df_corr = similarity_between_subjects(df)

In [ ]:
plt.figure(figsize=(15,5))
melted_df = pd.melt(df_z, id_vars=['subject'])
sns.boxplot(data=melted_df, x='value', y='variable', showfliers=False)
plt.xlabel('Z-score')
plt.ylabel('')
plt.title('Distribution of the graph measurements between participants')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
melted_df = pd.melt(, id_vars=['subject'])
sns.boxplot(data=melted_df, x='value', y='variable', showfliers=False)
plt.xlabel('Correlation')
plt.ylabel('')
plt.title('Correlation to the average measurements')
plt.show()

# Classtermap

In [ ]:
# Classtermap similarity between participants and measurements
sns.clustermap(df_z.iloc[:, 1:])
plt.show()

# CircosPlot

In [ ]:
# Structural connectivity average Cmap
G = make_graph(df.loc[0, 'Cmat'])
plot_graph_circos(G, sc_threshold=0.2)

In [ ]:
# Structural connectivity subject 26
G = make_graph(df.loc[22, 'Cmat'])
plot_graph_circos(G, sc_threshold=0.2)

# Minimum Spanning Trees

In [ ]:
freq_mst = np.zeros_like(df.loc[0, 'backbone'])
for i in range(1, 27):
    freq_mst += df.loc[i, 'backbone']

freq_mst /= 26

In [ ]:
plt.figure(figsize = (18, 7))
heatmap = sns.heatmap(freq_mst, cmap='coolwarm', cbar=True, square=True, 
                      cbar_kws={"label": "Probability of the node to be in the minimum spanning trees"})
plt.xlabel('Nodes')
plt.ylabel('Nodes')
plt.show()

In [ ]:
G = make_graph(freq_mst)
plot_graph_circos(G)

# Test models

In [ ]:
from neurolib.models.aln import ALNModel
model = ALNModel(Cmat=ds.Cmats[26], Dmat=ds.Dmats[26])
model.params['dt'] = 0.1
model.params['duration'] = 20 * 1000  # ms

# add custom parameter for downsampling results
# 10 ms sampling steps for saving data, should be multiple of dt
model.params['save_dt'] = 10.0
model.params["tauA"] = 600.0
model.params["sigma_ou"] = 0.0
model.params["b"] = 20.0

model.params["Ke_gl"] = 300.0
model.params["signalV"] = 80.0
model.params["mue_ext_mean"] = 1.5
model.params["mui_ext_mean"] = 0.2


In [ ]:
model.run()

In [ ]:
model.output.shape

In [ ]:
plt.plot(np.arange(0, 10000), model.output[56, 10000:10000*2])

In [ ]:
plt.figure(figsize=(12, 6))
plt.imshow(model.output[:, :10000], aspect='auto') # Aspect=1.2